In [104]:
from datetime import datetime, timedelta
import json
import logging
import traceback

# Set up logging
logging.basicConfig(level=logging.ERROR, filename='errors.log')

# MEETING_ID = 1
# MEETINGS_DB = {}

def retrive_calendar_events(user, start, end):
    global MEETING_ID
    events_list = []
    print(f'user: {user}')
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append(user)
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            EventId = event['id']
            
        except Exception as e:
            continue
        # MEETING_ID = MEETING_ID + 1
        events_list.append(
            {"StartTime" : start_time, 
             # "Id": MEETING_ID,
             "EndTime": end_time, 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"],
              "Id": EventId})
    return events_list
# def find_free_slots(schedules, start_time, end_time, duration_minutes, min_attendees=1):
#     """
#     Find free slots across multiple people's schedules
    
#     Args:
#         schedules: List of schedules, where each schedule is a list of meetings
#         start_time: Search start time as string '2025-07-18T00:00:00+05:30'
#         end_time: Search end time as string '2025-07-18T23:59:59+05:30'  
#         duration_minutes: Required duration in minutes
#         min_attendees: Minimum number of people that should be free (default: 1)
    
#     Returns:
#         List of free slots with start time, end time, and available attendees
#     """
    
#     # Parse input times
#     search_start = datetime.fromisoformat(start_time)
#     search_end = datetime.fromisoformat(end_time)
#     duration = timedelta(minutes=duration_minutes)
    
#     # Create a list to track all busy periods for each person
#     all_busy_periods = []
    
#     for person_idx, schedule in enumerate(schedules):
#         busy_periods = []
#         for meeting in schedule:
#             # Skip meetings that don't have meaningful time blocks (ignore summary)
#             meeting_start = datetime.fromisoformat(meeting['StartTime'])
#             meeting_end = datetime.fromisoformat(meeting['EndTime'])
            
#             # Only consider meetings within our search window
#             if meeting_end > search_start and meeting_start < search_end:
#                 # Clip to search window
#                 clipped_start = max(meeting_start, search_start)
#                 clipped_end = min(meeting_end, search_end)
#                 busy_periods.append((clipped_start, clipped_end))
        
#         # Sort and merge overlapping periods
#         busy_periods.sort()
#         merged_busy = []
#         for start, end in busy_periods:
#             if merged_busy and start <= merged_busy[-1][1]:
#                 # Overlapping, merge
#                 merged_busy[-1] = (merged_busy[-1][0], max(merged_busy[-1][1], end))
#             else:
#                 merged_busy.append((start, end))
        
#         all_busy_periods.append(merged_busy)
    
#     # Find time slots where at least min_attendees people are free
#     free_slots = []
#     current_time = search_start
#     time_increment = timedelta(minutes=15)  # Check every 15 minutes
    
#     while current_time + duration <= search_end:
#         slot_end = current_time + duration
        
#         # Count how many people are free during this slot
#         free_people = []
#         for person_idx, busy_periods in enumerate(all_busy_periods):
#             is_free = True
#             for busy_start, busy_end in busy_periods:
#                 # Check if slot overlaps with any busy period
#                 if not (slot_end <= busy_start or current_time >= busy_end):
#                     is_free = False
#                     break
            
#             if is_free:
#                 free_people.append(person_idx)
        
#         # If enough people are free, add this as a potential slot
#         if len(free_people) >= min_attendees:
#             free_slots.append({
#                 'start_time': current_time.isoformat(),
#                 'end_time': slot_end.isoformat(),
#                 'duration_minutes': duration_minutes,
#                 'available_people': free_people,
#                 'num_available': len(free_people)
#             })
#             return free_slots
        
#         current_time += time_increment
    
#     return free_slots

def extend_time_by_days(time_str: str, days: int = 7, return_as_str: bool = True) -> datetime:
    """
    Extends a datetime string by a given number of days.

    Args:
        time_str (str): The input time string in format "%Y-%m-%d %H:%M:%S"
        days (int): Number of days to add (default = 7)
        return_as_str (bool): If True, returns a string; else returns a datetime object.

    Returns:
        str | datetime: The extended time
    """
    try:
        # Try to parse as ISO format first (most common in your use case)
        try:
            dt = datetime.fromisoformat(time_str.replace('Z', '+00:00'))
        except ValueError:
            # Fall back to the original format
            dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
        
        extended_dt = dt + timedelta(days=days)

        assert isinstance(extended_dt, datetime), f"Expected datetime, got {type(extended_dt)}"
        
        return extended_dt
            
    except Exception as e:
        print(f"[ERROR] Failed to extend time: {e}")
        return None

def find_free_slots(schedules, start_time, end_time, duration_minutes, event_id: str="new_event"):
    """
    Optimized function to find free slots where ALL people are available
    
    Args:
        schedules: List of schedules, where each schedule is a list of meetings
        start_time: Search start time as string '2025-07-18T00:00:00+05:30'
        end_time: Search end time as string '2025-07-18T23:59:59+05:30'  
        duration_minutes: Required duration in minutes
    
    Returns:
        List of free slots with start time, end time
    """
    
    # Parse input times
    search_start = datetime.fromisoformat(start_time)
    search_end = datetime.fromisoformat(end_time)
    duration = timedelta(minutes=duration_minutes)
    # Collect ALL busy periods from ALL people into one list
    all_busy_periods = []

    for schedule in schedules:
        for meeting in schedule:
            meeting_start = datetime.fromisoformat(meeting['StartTime'])
            meeting_end = datetime.fromisoformat(meeting['EndTime'])
            # Only consider meetings within our search window
            if meeting_end > search_start and meeting_start < search_end:
                # Clip to search window
                clipped_start = max(meeting_start, search_start)
                clipped_end = min(meeting_end, search_end)
                all_busy_periods.append((clipped_start, clipped_end))
    
    # Sort all busy periods by start time
    all_busy_periods.sort()
    
    # Merge overlapping busy periods
    merged_busy = []
    for start, end in all_busy_periods:
        if merged_busy and start <= merged_busy[-1][1]:
            # Overlapping, merge by extending the end time
            merged_busy[-1] = (merged_busy[-1][0], max(merged_busy[-1][1], end))
        else:
            merged_busy.append((start, end))
    
    # Find free slots between busy periods
    free_slots = []
    current_time = search_start
    
    for busy_start, busy_end in merged_busy:
        # Check if there's a gap before this busy period
        if current_time + duration <= busy_start:
            # Found a potential free slot
            free_slots.append({
                'start_time': current_time.isoformat(),
                'end_time': (current_time + duration).isoformat(),
                'duration_minutes': duration_minutes,
                'event_id':event_id
            })
            return free_slots
        
        # Move current time to after this busy period
        current_time = max(current_time, busy_end)
    
    # Check if there's time after the last busy period
    if current_time + duration <= search_end:
        free_slots.append({
            'start_time': current_time.isoformat(),
            'end_time': (current_time + duration).isoformat(),
            'duration_minutes': duration_minutes,
            'event_id':event_id
        })
    
    return free_slots


def fetch_calendar_events(users: list[str], start_time: datetime, end_time: datetime):
    all_events = []
    # Fetch events for each user and append to the array
    for user in users:
        user_events = retrive_calendar_events(user, start_time, end_time)
        all_events.append(user_events)

    return all_events


def find_meeting_slots(users: list[str], start_time: datetime, end_time: datetime,
                       duration_minutes: int, priority: int=0,count: int =1,event_id: str="new_event"):
    """
    Find free meeting slots for multiple users
    
    Args:
        users (list): List of user email addresses
        start_time (str): Start time in ISO format (e.g., '2025-07-18T00:00:00+05:30')
        end_time (str): End time in ISO format (e.g., '2025-07-18T23:59:59+05:30')
        duration_minutes (int): Duration of meeting slots in minutes
        priority (int): 0 for low priority event and 1 for high priority_event
    
    Returns:
        list: List of available meeting slots
    """
    if count>=50:
        return []
    try:
        print('#1')
        start_time=start_time.isoformat()
        end_time=end_time.isoformat()
        print('#2')
        all_events = fetch_calendar_events(users, start_time, end_time)

        print('Got all events successfully')


        # Find free slots where all people are available
        free_slots = find_free_slots(
            schedules=all_events,
            start_time=start_time,
            end_time=end_time,
            duration_minutes=duration_minutes,
            event_id =event_id
        )
    
        if free_slots:
            print(f'Found free slot: {free_slots}')
            return free_slots
        print('\nNot found free slot\n')
    
        if priority == 0:
            print('\nchecking for next 7 days since its a low priority meeting\n')
            extended_end_time = extend_time_by_days(end_time)
            print(extended_end_time)
            return find_meeting_slots(users,datetime.fromisoformat(end_time),extended_end_time,duration_minutes,0,count+1,event_id)

        if priority == 1:
            print('\nhigh priority meeting; trying to reschedule\n') 
            free_slots = schedule_high_priority_meeting(schedules=all_events,
            start_time=start_time,
            end_time=end_time,
            duration_minutes=duration_minutes,event_id=event_id)

        if not free_slots:
            extended_end_time = extend_time_by_days(end_time)
            print(f'extended_end_time: {extended_end_time}')
            return find_meeting_slots(users,datetime.fromisoformat(end_time),extended_end_time,duration_minutes,0,count+1,event_id)
        print(f'\nFound free slot: {free_slots}\n')
        
        return free_slots
    except Exception as e:
        print(f'\nAn error occurred while finding free_slot: {e}\n')
        logging.error("An error occurred", exc_info=True)
        return []


In [98]:
def fetch_priority(summary: str):
    high_priority_keywords = [# Urgency
    "urgent", "emergency", "asap", "immediate", "critical", "high priority",
    "top priority", "time-sensitive", "escalate", "escalation", "blocker",

    # Client / External
    "client", "customer", "partner", "stakeholder", "vendor", "external",
    "investor", "cxo", "ceo", "vp", "leadership",

    # Escalation / Issue
    "incident", "outage", "production issue", "prod issue", "downtime",
    "rca", "root cause analysis", "follow-up on issue", "bug", "sev1", "sev2",

    # Business-Critical
    "launch", "release", "go-live", "deadline", "okr", "kpi", "qbr",
    "revenue", "billing", "compliance", "audit", "legal",

    # Decision-Making
    "decision", "approval", "sign-off", "final review", "planning",
    "strategy", "alignment",

    # Technical
    "deployment", "rollout", "integration", "performance review",
    "architecture", "hotfix", "patch"]
    for keyword in high_priority_keywords:
        if keyword in summary.lower():
            return 1 # high priority
    return 0 # low priority

# fetch_priority('hi team, lets schedule a Client meeting')

1

In [99]:
def schedule_high_priority_meeting(schedules, start_time, end_time, duration_minutes,event_id, priority=1):
    """
    Schedule a high priority meeting by canceling low priority events
    Assumes no free slots are available and we need to cancel events to make room
    
    Args:
        schedules: List of schedules, where each schedule is a list of meetings (with priority field)
        start_time: Search start time as string '2025-07-18T00:00:00+05:30'
        end_time: Search end time as string '2025-07-18T23:59:59+05:30'  
        duration_minutes: Required duration in minutes
        priority: Priority of the meeting being scheduled (0=low, 1=high)
    
    Returns:
        dict: Dictionary containing:
            - 'scheduled_slot': Time slot for the new meeting (if successful)
            - 'canceled_events': List of events that were canceled
            - 'success': Boolean indicating if scheduling was successful
            - 'message': Description of what happened
    """
    
    # Parse input times
    search_start = datetime.fromisoformat(start_time)
    search_end = datetime.fromisoformat(end_time)
    duration = timedelta(minutes=duration_minutes)
    
    # Collect all meetings within the search window
    all_meetings = []
    
    for schedule_idx, schedule in enumerate(schedules):
        for meeting_idx, meeting in enumerate(schedule):
            meeting_start = datetime.fromisoformat(meeting['StartTime'])
            meeting_end = datetime.fromisoformat(meeting['EndTime'])
            
            # Only consider meetings within our search window
            if meeting_end > search_start and meeting_start < search_end:
                all_meetings.append({
                    'meeting': meeting,
                    'start': meeting_start,
                    'end': meeting_end,
                    'priority': fetch_priority(meeting.get('Summary')),
                    'duration_minutes': int((meeting_end - meeting_start).total_seconds() / 60),
                })
    
    # Sort meetings by start time
    all_meetings.sort(key=lambda x: x['start'])
    
    # Only consider canceling meetings with priority 0 (low priority)
    cancelable_meetings = [m for m in all_meetings if m['priority'] == 0]
    
    if not cancelable_meetings:     
        return []
    
    # Try to cancel low priority events first
    return _cancel_low_priority_events(cancelable_meetings, all_meetings, search_start, search_end, duration)

def _cancel_low_priority_events(cancelable_meetings, all_meetings, search_start, search_end, duration, event_id):
    """
    Try to cancel low priority events to make room for high priority meeting
    """
    from itertools import combinations
    
    print(f"Found {len(cancelable_meetings)} cancelable low priority meetings:")
    for meeting in cancelable_meetings:
        meeting = meeting.get('meeting')
        print(meeting)
    
    # Strategy 1: Try to find a single meeting that can be replaced
    for meeting in cancelable_meetings:
        meeting_duration = meeting['end'] - meeting['start']
        print(meeting_duration, duration)
        # Check if this meeting is long enough to fit our new meeting
        if meeting_duration >= duration:
            # Calculate the best position for the new meeting within this slot
            proposed_start = max(meeting['start'], search_start)
            proposed_end = proposed_start + duration
            
            # Make sure it fits within the canceled meeting's time and search window
            if (proposed_end <= meeting['end'] and 
                proposed_end <= search_end and 
                proposed_start >= search_start):
                print("#####")
                free_slots=schedule_canceled_events([{
                        'meeting': meeting,
                    }])
                print("########################")
                print(free_slots)
                free_slots.append({
                        'start_time': proposed_start.isoformat(),
                        'end_time': proposed_end.isoformat(),
                        'duration_minutes': int(duration.total_seconds() / 60),
                        'event_id': event_id
                    })
                return free_slots
    
    # Strategy 2: Try combinations of multiple meetings
    for r in range(2, min(len(cancelable_meetings) + 1, 5)):  # Try up to 4 meetings
        for meetings_to_cancel in combinations(cancelable_meetings, r):
            print('#3')
            # Calculate total time that would be freed up
            total_freed_time = timedelta(0)
            earliest_start = min(m['start'] for m in meetings_to_cancel)
            latest_end = max(m['end'] for m in meetings_to_cancel)
            print('#4')
            # Check if the meetings overlap or are close enough to create a continuous slot
            sorted_meetings = sorted(meetings_to_cancel, key=lambda x: x['start'])
            print('#5')
            # Check for gaps between meetings
            has_suitable_slot = False
            current_start = max(earliest_start, search_start)
            print('#6')
            for i in range(len(sorted_meetings)):
                # Check if we can fit the meeting starting from current position
                if i == 0:
                    print('#7')
                    slot_start = max(sorted_meetings[i]['start'], search_start)
                else:
                    print('#8')
                    # Check gap between previous meeting end and current meeting start
                    gap_start = sorted_meetings[i-1]['end']
                    gap_end = sorted_meetings[i]['start']
                    if gap_end - gap_start >= duration:
                        slot_start = gap_start
                    else:
                        slot_start = max(sorted_meetings[i]['start'], search_start)
                
                slot_end = slot_start + duration
                print('#9')
                # Check if this slot fits within bounds
                if (slot_end <= min(latest_end, search_end) and 
                    slot_start >= search_start):
                    has_suitable_slot = True
                    print('#10')
                    canceled_events = []
                    for meeting in meetings_to_cancel:
                        canceled_events.append({
                            'meeting': meeting['meeting'],
                        })
                    print('#11')
                    free_slots=schedule_canceled_events(canceled_events)
                    print('#12')
                    free_slots.append({
                        'start_time': proposed_start.isoformat(),
                        'end_time': proposed_end.isoformat(),
                        'duration_minutes': int(duration.total_seconds() / 60),
                        'event_id':event_id
                    })
                    return free_slots
                    
            
            if has_suitable_slot:
                break
    
    return []
    
def schedule_canceled_events(canceled_events):
    free_slots=[]
    for event in canceled_events:
        print(f'#####: {event}')
        meeting=event['meeting']['meeting']
        start_time=meeting['StartTime']
        end_time=meeting['EndTime']
        users=meeting["Attendees"]
        event_id=meeting['Id']
        meeting_start = datetime.fromisoformat(start_time)
        meeting_end = datetime.fromisoformat(end_time)
        duration = int((meeting_end - meeting_start).total_seconds() / 60)
        extended_end_time = extend_time_by_days(end_time)
        free_slot=find_meeting_slots(users,datetime.fromisoformat(end_time),extended_end_time,duration,0,1,event_id)
        free_slots.append(free_slot[0])
    return free_slots
    